Platfrom: ArcGIS 10.8
Enviroment: Python 2.7

In [2]:
import arcpy
from arcpy import env
from arcpy.sa import *
import os
import math
import shutil
import gc

In [12]:
root = r'E:\_2023年\[研究论文]\[中国城乡流域]\[data2]'

In [13]:
data_path = os.path.join(root, 'Step1_data')

In [14]:
arcpy.CheckOutExtension("spatial")
arcpy.gp.overwriteOutput = 1
arcpy.env.overwriteOutput = 1
env.workspace = data_path

In [15]:
# WorldPop2018_China = Raster(data_path + r'\Population.tif')
WorldPop2018_China = Raster(data_path + r'\pop2018Int.tif')

In [16]:
arcpy.env.cellSize = WorldPop2018_China
arcpy.env.snapRaster = WorldPop2018_China
arcpy.env.extent = WorldPop2018_China
sr = arcpy.Describe(WorldPop2018_China).spatialReference
arcpy.env.outputCoordinateSystem = sr

In [17]:
Landuse_1km_China = Raster(data_path + r'\LUCC_1km.tif')
DEM_1km_China = Raster(data_path + r'\DEM.tif')
Slope_1km_China = Raster(data_path + r"\Slope.tif")

In [18]:
GUB2018_China = data_path + r'\GUB_china_2018_1125.shp'

In [19]:
# Step2: ZonalStatistics
Step2_Path = root + r'\Step2_ZoneStat'

In [20]:
if not os.path.exists(Step2_Path):
    os.mkdir(Step2_Path)
else:
    shutil.rmtree(Step2_Path)
    os.mkdir(Step2_Path)

In [21]:
outZonalStatistics = Int(ZonalStatistics(GUB2018_China, "FID", WorldPop2018_China, "SUM", "DATA"))
outZonalStatistics.save(Step2_Path + r'\GUB_POP_1km_China.tif')

In [22]:
# step3: class1(-8)
Step3_Path = root + r'\Step3_class1(-6)'

In [23]:
if not os.path.exists(Step3_Path):
    os.mkdir(Step3_Path)
else:
    shutil.rmtree(Step3_Path)
    os.mkdir(Step3_Path)

In [24]:
class1 = Con((outZonalStatistics >= 10000000), 1)
class1.save(Step3_Path + r'\class1.tif')
class2 = Con((outZonalStatistics >= 5000000) & (outZonalStatistics < 10000000), 2)
class2.save(Step3_Path + r'\class2.tif')
class3 = Con((outZonalStatistics >= 1000000) & (outZonalStatistics < 5000000), 3)
class3.save(Step3_Path + r'\class3.tif')
class4 = Con((outZonalStatistics >= 500000) & (outZonalStatistics < 1000000), 4)
class4.save(Step3_Path + r'\class4.tif')
class5 = Con((outZonalStatistics >= 50000) & (outZonalStatistics < 500000), 5)
class5.save(Step3_Path + r'\class5.tif')
class6 = Con((outZonalStatistics >= 20000) & (outZonalStatistics < 50000), 6)
class6.save(Step3_Path + r'\class6.tif')

In [25]:
# step4: OSM Buffer
Step4_Path = root + r'\Step4_OSM_Buffer'

In [26]:
if not os.path.exists(Step4_Path):
    os.mkdir(Step4_Path)
else:
    shutil.rmtree(Step4_Path)
    os.mkdir(Step4_Path)

In [27]:
roads_primary = data_path + r'\roads_primary.shp'
roads_secondary = data_path + r'\roads_secondary.shp'
roads_tertiary = data_path + r'\roads_tertiary.shp'
roads_motorway = data_path + r'\roads_motorway.shp'
roads_trunk = data_path + r'\roads_trunk.shp'

railways_rail = data_path + r'\railways_rail.shp'
high_speed_rail = data_path + r'\HighSpeedRail.shp'
high_speed_rail_point = data_path + r'\HighSpeedRail_P.shp'

ocean = data_path + r'\Ocean.shp'

In [28]:
arcpy.Buffer_analysis(high_speed_rail_point, Step4_Path + r'\rail_point_buffer25.shp', '0.02 DecimalDegrees', "FULL", "ROUND", "LIST")
rail_point_buffer = os.path.join(Step4_Path, 'rail_point_buffer25.shp')
arcpy.AddField_management(rail_point_buffer, 'cost', 'FLOAT')

arcpy.Buffer_analysis(high_speed_rail, Step4_Path + r'\rail_buffer15.shp', '0.015 DecimalDegrees', "FULL", "FLAT", "LIST")
rail_buffer = os.path.join(Step4_Path, 'rail_buffer15.shp')
arcpy.AddField_management(rail_buffer, 'cost', 'FLOAT')

<Result 'E:\\_2023年\\[研究论文]\\[中国城乡流域]\\[data2]\\Step4_OSM_Buffer\\rail_buffer15.shp'>

In [29]:
# Step5: Cost
Step5_Path = root + r'\Step5_Cost'

In [30]:
if not os.path.exists(Step5_Path):
    os.mkdir(Step5_Path)
else:
    shutil.rmtree(Step5_Path)
    os.mkdir(Step5_Path)

In [31]:
# roads_primary(80km/h)
arcpy.CalculateField_management(roads_primary, 'cost', 80, "PYTHON_9.3")
arcpy.PolylineToRaster_conversion(roads_primary, "cost", Step5_Path + r"\roads_primary_cost.tif", "MAXIMUM_LENGTH",
                                  "NONE", WorldPop2018_China)
# roads_secondary(60km/h)
arcpy.CalculateField_management(roads_secondary, 'cost', 60, "PYTHON_9.3")
arcpy.PolylineToRaster_conversion(roads_secondary, "cost", Step5_Path + r"\roads_secondary_cost.tif", "MAXIMUM_LENGTH",
                                  "NONE", WorldPop2018_China)
# roads_tertiary(40km/h)
arcpy.CalculateField_management(roads_tertiary, 'cost', 40, "PYTHON_9.3")
arcpy.PolylineToRaster_conversion(roads_tertiary, "cost", Step5_Path + r"\roads_tertiary_cost.tif", "MAXIMUM_LENGTH",
                                  "NONE", WorldPop2018_China)
# roads_trunk(90km/h)
arcpy.CalculateField_management(roads_trunk, 'cost', 90, "PYTHON_9.3")
arcpy.PolylineToRaster_conversion(roads_trunk, "cost", Step5_Path + r"\roads_trunk_cost.tif", "MAXIMUM_LENGTH", "NONE", WorldPop2018_China)
# roads_motorway(120km/h)
arcpy.CalculateField_management(roads_motorway, 'cost', 120, "PYTHON_9.3")
arcpy.PolylineToRaster_conversion(roads_motorway, "cost", Step5_Path + r"\roads_motorway_cost.tif", "MAXIMUM_LENGTH",
                                  "NONE", WorldPop2018_China)
# railways_rail(200km/h)
arcpy.CalculateField_management(high_speed_rail, 'cost', 200, "PYTHON_9.3")
arcpy.PolylineToRaster_conversion(railways_rail, "cost", Step5_Path + r"\railways_rail_cost.tif",
                                  "MAXIMUM_LENGTH",
                                  "NONE", WorldPop2018_China)
# HighSpeedRail(250km/h)
arcpy.CalculateField_management(high_speed_rail, 'cost', 250, "PYTHON_9.3")
arcpy.PolylineToRaster_conversion(high_speed_rail, "cost", Step5_Path + r"\high_speed_rail_cost.tif",
                                  "MAXIMUM_LENGTH",
                                  "NONE", WorldPop2018_China)

# HighSpeedRail_Buffer
arcpy.CalculateField_management(rail_buffer, 'cost', 1, "PYTHON_9.3")  # 高铁buffer 1km/h
arcpy.PolygonToRaster_conversion(rail_buffer, "cost", Step5_Path + r"\hsr_bf_cost15.tif", "CELL_CENTER", "NONE", WorldPop2018_China)

arcpy.CalculateField_management(rail_point_buffer, 'cost', 5, "PYTHON_9.3")
arcpy.PolygonToRaster_conversion(rail_point_buffer, "cost", Step5_Path + r"\hsr_p_bf_cost25.tif", "CELL_CENTER", "NONE", WorldPop2018_China)  # 看看cellsize合理性

# 轮渡
arcpy.CalculateField_management(ocean, 'cost', 19, "PYTHON_9.3")
arcpy.PolygonToRaster_conversion(ocean, "cost", Step5_Path + r"\Ocean_cost.tif", "CELL_CENTER", "NONE", WorldPop2018_China)

<Result 'E:\\_2023年\\[研究论文]\\[中国城乡流域]\\[data2]\\Step5_Cost\\Ocean_cost.tif'>

In [32]:
ElevationFactor = 1.016 * Exp((-0.0001072) * DEM_1km_China)
ElevationFactor.save(Step5_Path + r'\ElevationFactor.tif')

In [33]:
WalkingSpeed = 6 * Exp((-3.5) * Abs(Tan(0.01745 * Slope_1km_China) + 0.05))
WalkingSpeed.save(Step5_Path + r'\WalkingSpeed.tif')
SlopeFactor = WalkingSpeed / 5.0
SlopeFactor.save(Step5_Path + r'\SlopeFactor.tif')

In [34]:
# MODIS_LUCC Code
'''
1: 常绿针叶林 3.24
2：常绿阔叶林 1.62
3：落叶针叶林 3.24
4：落叶阔叶林 4.00
5：混交林 3.24
6：郁闭灌木丛 3.00
7：有林草地 4.20
8：稀树草原 4.86
10：草地 4.86
11：永久湿地 2.00
12：农田 2.50
13：城镇与建成区 5.00
14：农田与自然植被镶嵌体 3.24
15：冰雪 1.62
16：裸地 3.00
17：水体 1.00
'''
LUCC_Volocity = Reclassify(Landuse_1km_China, "Value",
                           RemapValue([[1, 324], [2, 162], [3, 324], [4, 400], [5, 324],
                                       [6, 300], [7, 420], [8, 486], [9, 200], [10, 486],
                                       [11, 200], [12, 250], [13, 500], [14, 324], [15, 162],
                                       [16, 300], [17, 100]]))
LUCC_Volocity.save(Step5_Path + r'\LUCC_Velocity.tif')

In [35]:
env.workspace = Step5_Path
LUCC_Weight_Velocity = arcpy.ia.RasterCalculator([LUCC_Volocity,SlopeFactor,ElevationFactor],["x", "y", "z"], "x*y*z*0.01", "IntersectionOf", "FirstOf")
LUCC_Weight_Velocity.save(Step5_Path + r'\LUCC_Weight_Velocity.tif')

In [36]:
# Friction(roads and land cover)
# 第一步镶嵌： 道路+路面 MAXIMUM
arcpy.MosaicToNewRaster_management("roads_primary_cost.tif; roads_secondary_cost.tif; roads_tertiary_cost.tif;"
                                   "roads_motorway_cost.tif; roads_trunk_cost.tif; railways_rail_cost.tif;"
                                   "LUCC_Weight_Velocity.tif; Ocean_cost.tif",
                                   Step5_Path, "Mosaic1.tif", sr,
                                   "32_BIT_FLOAT", "", "1", "MAXIMUM",
                                   "FIRST")
# 第二步镶嵌： 叠加高铁缓冲区 MINIMUM
arcpy.MosaicToNewRaster_management("Mosaic1.tif; hsr_bf_cost15.tif",
                                   Step5_Path, "Mosaic2.tif", sr,
                                   "32_BIT_FLOAT", "", "1", "MINIMUM",
                                   "FIRST")
# 第三步镶嵌： 叠加高铁 MAXIMUM
arcpy.MosaicToNewRaster_management("Mosaic2.tif; high_speed_rail_cost.tif",
                                   Step5_Path, "Mosaic3.tif", sr,
                                   "32_BIT_FLOAT", "", "1", "MAXIMUM",
                                   "FIRST")
# 第四步镶嵌：叠加站点 MINIMUM
# 最终获得 速度栅格 Velocity.tif
arcpy.MosaicToNewRaster_management("Mosaic3.tif; hsr_p_bf_cost25.tif",
                                   Step5_Path, "Velocity.tif", sr,
                                   "32_BIT_FLOAT", "", "1", "MINIMUM",
                                   "FIRST")

<Result 'E:\\_2023年\\[研究论文]\\[中国城乡流域]\\[data2]\\Step5_Cost\\Velocity.tif'>

In [37]:
Velocity = Raster(Step5_Path + r'\Velocity.tif')
# Cost Raster
Cost = 1 / Velocity
Cost.save(Step5_Path + r"\Cost.tif")

In [38]:
# Step6: class_traveltime
Step6_Path = root + r'\Step6_class_traveltime'

In [39]:
if not os.path.exists(Step6_Path):
    os.mkdir(Step6_Path)
else:
    shutil.rmtree(Step6_Path)
    os.mkdir(Step6_Path)

In [40]:
arcpy.env.parallelProcessingFactor = "0"
arcpy.env.outputCoordinateSystem = 'PROJCS["Albers_Conic_Equal_Area",\
GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],\
PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["false_easting",0.0],\
PARAMETER["false_northing",0.0],PARAMETER["central_meridian",105.0],PARAMETER["standard_parallel_1",25.0],\
PARAMETER["standard_parallel_2",47.0],PARAMETER["latitude_of_origin",0.0],UNIT["Meter",1.0]]'

outCostDist = CostDistance(os.path.join(Step3_Path, "class1.tif"), Cost)
City1_Cost = outCostDist / 1000
City1_Cost.save(Step6_Path + r"\travel_time_to_cities_1.tif")

outCostDist = CostDistance(os.path.join(Step3_Path, "class2.tif"), Cost)
City2_Cost = outCostDist / 1000
City2_Cost.save(Step6_Path + r"\travel_time_to_cities_2.tif")

outCostDist = CostDistance(os.path.join(Step3_Path, "class3.tif"), Cost)
City3_Cost = outCostDist / 1000
City3_Cost.save(Step6_Path + r"\travel_time_to_cities_3.tif")

outCostDist = CostDistance(os.path.join(Step3_Path, "class4.tif"), Cost)
City4_Cost = outCostDist / 1000
City4_Cost.save(Step6_Path + r"\travel_time_to_cities_4.tif")

outCostDist =CostDistance(os.path.join(Step3_Path, "class5.tif"), Cost)
City5_Cost = outCostDist / 1000
City5_Cost.save(Step6_Path + r"\travel_time_to_cities_5.tif")

outCostDist = CostDistance(os.path.join(Step3_Path, "class6.tif"), Cost)
City6_Cost = outCostDist / 1000
City6_Cost.save(Step6_Path + r"\travel_time_to_cities_6.tif")

In [42]:
del City1_Cost, City2_Cost, City3_Cost, City4_Cost, City5_Cost, City6_Cost

NameError: name 'City1_Cost' is not defined

In [43]:
arcpy.env.outputCoordinateSystem = "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]]"
arcpy.env.parallelProcessingFactor = ""

In [46]:
# Step6_proj: class_traveltime project
Step6_proj_Path = root + r'\Step6_class_traveltime\traveltime_project'

In [47]:
if not os.path.exists(Step6_proj_Path):
    os.mkdir(Step6_proj_Path)
else:
    shutil.rmtree(Step6_proj_Path)
    os.mkdir(Step6_proj_Path)

In [48]:
for i in range(1,7):
    filename = "travel_time_to_cities_"+ str(i) +".tif"
    arcpy.ProjectRaster_management(in_raster=os.path.join(Step6_Path, filename), 
                                   out_raster=os.path.join(Step6_proj_Path, filename), 
                                   out_coor_system="GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]]", 
                                   resampling_type="BILINEAR", cell_size="0.008333333333 0.008333333333", 
                                   geographic_transform="", Registration_Point="", 
                                   in_coor_system="PROJCS['Albers_Conic_Equal_Area',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],\
                                   PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Albers'],PARAMETER['false_easting',0.0],PARAMETER['false_northing',0.0],\
                                   PARAMETER['central_meridian',105.0],PARAMETER['standard_parallel_1',25.0],PARAMETER['standard_parallel_2',47.0],PARAMETER['latitude_of_origin',0.0],UNIT['Meter',1.0]]", 
                                   vertical="NO_VERTICAL")

In [49]:
# Step7: class_traveltime_code
Step7_Path = root + r'\Step7_class_traveltime_reclass'

In [50]:
if not os.path.exists(Step7_Path):
    os.mkdir(Step7_Path)
else:
    shutil.rmtree(Step7_Path)
    os.mkdir(Step7_Path)

In [51]:
rasterstype30 = ['travel_time_to_cities_1', 'travel_time_to_cities_2', 'travel_time_to_cities_3',
                 'travel_time_to_cities_4', 'travel_time_to_cities_5', 'travel_time_to_cities_6']

In [52]:
# Set up for travel masks for distances 0 to 30 minutes from cities - peri-urban
outfolder = Step7_Path + r'\tt_0to30min'
if not os.path.exists(outfolder):
    os.mkdir(outfolder)
inFalseRaster = [7, 8, 9, 10, 11, 22]
# Loop through rasterstype30 for peri-urban travel masks
for i, raster in enumerate(rasterstype30):
    print(raster)
    outSetNull = SetNull(os.path.join(Step6_proj_Path, raster + '.tif'), inFalseRaster[i], "value > 0.5")
    print(inFalseRaster[i])
    outras = os.path.join(outfolder, raster + "_tt1.tif")
    outSetNull.save(outras)

travel_time_to_cities_1
7
travel_time_to_cities_2
8
travel_time_to_cities_3
9
travel_time_to_cities_4
10
travel_time_to_cities_5
11
travel_time_to_cities_6
22


In [53]:
inFalseRaster = [12, 13, 14, 15, 16, 23]

# Set up for travel masks for distances 30 to 60 minutes from cities - peri-urban
outfolder = Step7_Path + r'\tt_30to60min'
if not os.path.exists(outfolder):
    os.mkdir(outfolder)
# Loop through rasterstype30 for peri-urban travel masks
for i, raster in enumerate(rasterstype30):
    print(raster)
    outSetNull = SetNull(os.path.join(Step6_proj_Path, raster + '.tif'), inFalseRaster[i], "value <= 0.5 or value >1")
    print(inFalseRaster[i])
    outras = os.path.join(outfolder, raster + "_tt2.tif")
    outSetNull.save(outras)

travel_time_to_cities_1
12
travel_time_to_cities_2
13
travel_time_to_cities_3
14
travel_time_to_cities_4
15
travel_time_to_cities_5
16
travel_time_to_cities_6
23


In [54]:
inFalseRaster = [17, 18, 19, 20, 21, 24]

# Set up for travel masks for distances 60 to 120 minutes from cities - peri-urban
outfolder = Step7_Path + r'\tt_60to120min'
if not os.path.exists(outfolder):
    os.mkdir(outfolder)

# Loop through rasterstype30 for peri-urban travel masks
for i, raster in enumerate(rasterstype30):
    print(raster)
    outSetNull = SetNull(os.path.join(Step6_proj_Path, raster + '.tif'), inFalseRaster[i], "value <= 1 or value > 2")
    print(inFalseRaster[i])
    outras = os.path.join(outfolder, raster + "_tt3.tif")
    outSetNull.save(outras)

travel_time_to_cities_1
17
travel_time_to_cities_2
18
travel_time_to_cities_3
19
travel_time_to_cities_4
20
travel_time_to_cities_5
21
travel_time_to_cities_6
24


In [55]:
inFalseRaster = [25, 26, 27, 28, 29, 30]
# Set up for travel masks for distances 120 to 180 minutes from cities - peri-urban
outfolder = Step7_Path + r'\tt_120to180min'
if not os.path.exists(outfolder):
    os.mkdir(outfolder)

# Loop through rasterstype30 for peri-urban travel masks
for i, raster in enumerate(rasterstype30):
    print(raster)
    outSetNull = SetNull(os.path.join(Step6_proj_Path, raster + '.tif'), inFalseRaster[i], "value <= 2 or value >3")
    print(inFalseRaster[i])
    outras = os.path.join(outfolder, raster + "_tt4.tif")
    outSetNull.save(outras)

travel_time_to_cities_1
25
travel_time_to_cities_2
26
travel_time_to_cities_3
27
travel_time_to_cities_4
28
travel_time_to_cities_5
29
travel_time_to_cities_6
30


In [56]:
outfolder = Step7_Path + r'\Hinterland'
if not os.path.exists(outfolder):
    os.mkdir(outfolder)
outSetNull = SetNull(os.path.join(Step6_proj_Path, raster + '.tif'), 31, "value < 0")
outras = os.path.join(outfolder, "Hinterland.tif")
outSetNull.save(outras)

In [57]:
del outSetNull

In [58]:
# Step8: final output
Step8_Path = root + r'\Step8_final_output'

In [59]:
if not os.path.exists(Step8_Path):
    os.mkdir(Step8_Path)
else:
    shutil.rmtree(Step8_Path)
    os.mkdir(Step8_Path)

In [60]:
import glob
os.chdir(Step7_Path)
file_list = []
for file in glob.glob(r'.\**\*.tif'):
    file_list.append(Step7_Path + file.split('.')[1]+'.tif')

In [61]:
os.chdir(Step3_Path)
for file in glob.glob(r'.\*.tif'):
    file_list.append(Step3_Path +file.split('.')[1]+'.tif')

In [62]:
arcpy.MosaicToNewRaster_management('; '.join(file_list),
                                   Step8_Path, "URCA2018_China.tif", None,
                                   "8_BIT_UNSIGNED", None, "1", "MINIMUM",
                                   "FIRST")

<Result 'E:\\_2023年\\[研究论文]\\[中国城乡流域]\\[data2]\\Step8_final_output\\URCA2018_China.tif'>